In [2]:
import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import mpra_tools.predicted_occupancy as po
import mpra_tools.fasta_utils as fu
import math
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import random
from mpra_tools.fasta_utils import write_fasta

In [9]:
activity_df = pd.read_csv("Data/activity.csv", index_col=0)
retinopathy_df = pd.read_csv("Data/retinopathy_reformatted.txt", sep='\t', index_col=1)
L = 164

### Create fasta file for retinopathy data

In [5]:
# ret_s = pd.Series(
#     data = retinopathy_df['sequence'],
#     index=retinopathy_df.index
# )
# write_fasta(ret_s, "Data/Fasta/retinopathy.fasta")